# Compile constituency parse info

(one-time)

In [1]:
import sys, os, json, string
import spacy
import numpy as np
import pandas as pd
from collections import defaultdict
from benepar.spacy_plugin import BeneparComponent

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(BeneparComponent('benepar_en'))

Instructions for updating:
non-resource variables are not supported in the long term


In [125]:
# with open('data/moby-passages-36/item_to_passage_final.json', 'w') as f:
#     json.dump(item_to_passage_cleaned, f, indent=2)

In [138]:
with open('data/moby-passages-36/item_to_passage_final.json') as f:
    item_to_passage_cleaned = json.load(f)
item_to_doc = {}
for item, passage in item_to_passage_cleaned.items():
#     print('processing', item)
    __doc = nlp(passage)
    item_to_doc[item] = __doc

In [139]:
item_to_recstring_list = {}
with open('data/moby-passages-36/passages-with-line-break-and-recstring.tsv') as f:
    next(f)
    for line in f:
        _, _, item, _, recstring = line.strip().split('\t')
        item_to_recstring_list[item] = recstring.split()

In [146]:
item_to_doc_id_to_recstring_id = {}
for item, doc in item_to_doc.items():
#     print('\n\n\n', item)
    doc_id_to_recstring = {}
    curr_idx = 0
    matched = 0
    for idx, x in enumerate(doc):
        if all(y in string.punctuation for y in str(x)):
            continue
        elif ((str(x)[0] not in {'"'} and any(y in string.punctuation for y in str(x)))\
                and (str(x) not in {'Mrs.', 'Ms.', 'Dr.'}))\
                or (str(x) == 'not' and str(doc[idx - 1]) == 'can'):
            assert curr_idx > 0
            curr_idx -= 1
        doc_id_to_recstring[x.i] = curr_idx
        if str(x).lower() == item_to_recstring_list[item][curr_idx]:
            matched += 1
        
        curr_idx += 1
    assert 1 + max(v for k, v in doc_id_to_recstring.items()) == len(item_to_recstring_list[item])
    print(item, 'matched', matched / len(item_to_recstring_list[item]))
    item_to_doc_id_to_recstring_id[item] = doc_id_to_recstring

310 matched 1.0
320 matched 0.9662921348314607
330 matched 1.0
350 matched 1.0
410 matched 0.9571428571428572
420 matched 1.0
450 matched 0.9540229885057471
520 matched 0.9871794871794872
1401 matched 0.9487179487179487
1502 matched 0.9836065573770492
1505 matched 1.0
1603 matched 1.0
1703 matched 0.9402985074626866
1804 matched 1.0
1807 matched 0.9861111111111112
1903 matched 0.9868421052631579
2001 matched 1.0
2101 matched 1.0
2102 matched 0.9866666666666667
2201 matched 0.9767441860465116
2202 matched 1.0
2203 matched 1.0
2302 matched 1.0
2401 matched 0.9876543209876543
2402 matched 1.0
2403 matched 0.9746835443037974
2503 matched 1.0
2504 matched 0.9466666666666667
2601 matched 0.9572649572649573
2701 matched 0.9743589743589743
2801 matched 0.9659090909090909
2803 matched 0.9834710743801653
2902 matched 0.9741379310344828
2904 matched 1.0
3105 matched 0.9794520547945206
3106 matched 1.0


In [148]:
def bottom_up_representation(sent):
    constituent_to_depth = {}
    token_to_parents = {}
    for token in sent:
        curr = token
        depth = 1
        parents = set()
        while curr._.parent:
            curr = curr._.parent
            parents.add(curr)
            if curr not in constituent_to_depth:
                constituent_to_depth[curr] = depth
            constituent_to_depth[curr] = max(depth, constituent_to_depth[curr])
            depth += 1
        token_to_parents[token] = parents
    return constituent_to_depth, token_to_parents

def bottom_up_feature_vector(sent, k=1):
    rv = [0]
    constituent_to_depth, token_to_parents = bottom_up_representation(sent)
    for idx, token in enumerate(sent):
        if idx == 0:
            continue
        prev_token = sent[idx - 1]
        
        common_ancestors = token_to_parents[token] & token_to_parents[prev_token]
        common_ancestors_depths = set(constituent_to_depth[anc] for anc in common_ancestors)
        
        if k in common_ancestors_depths:
            rv.append(1)
        else:
            rv.append(0)
    return rv

def pretty_print_sisters_bottom_up(sent, k):
    x = bottom_up_feature_vector(sent, k)
    print(x)
    for idx, val in enumerate(x):
        if val == 0:
            print('\n' + str(sent[idx]), end=' ')
        else:
            print(sent[idx], end=' ')

In [149]:
def get_sentence_depth(sentence):
    k = 20
    while True:
        feature_vector = bottom_up_feature_vector(sentence, k)
        assert sum(feature_vector) in {0, len(sentence) - 1}
        if sum(feature_vector) == len(sentence) - 1:
            return k
        k -= 1

In [93]:
# wow this logic was tricky

MAX_DEPTH = 30
for item, doc in item_to_doc.items():
    sentences = list(doc.sents)
    for sentence in sentences:
        sentence_depth = get_sentence_depth(sentence)
        
        for k in range(1, sentence_depth + 1):
            feature_vector = list(reversed(bottom_up_feature_vector(sentence, k)))
            reversed_sentence = reversed(sentence)
            token = next(reversed_sentence)
            prev = feature_vector[0]
            if prev == 0:
                clusters = [[]]
            else:
                clusters = [[token]]
            
            for val in feature_vector[1:]:
                token = next(reversed_sentence)
                if val == 1:
                    clusters[-1].append(token)
                if val == 0:
                    if prev == 1:
                        clusters[-1].append(token)
                    clusters.append([])
                prev = val
            clusters = [x[::-1] for x in clusters if x][::-1]
            assert all(len(x) > 1 for x in clusters)
            
            for cluster in clusters:
                if not str(cluster[0])[0].isalpha():
                    cluster = cluster[1:]
                print(cluster)
#         break

#     break

[a, pie]
[bake, a, pie]
[to, bake, a, pie]
[wanted, to, bake, a, pie]
[Meg, wanted, to, bake, a, pie, .]
[her, mother]
[for, help]
[asked, her, mother, for, help]
[She, asked, her, mother, for, help, .]
[the, flour, ,, sugar, ,, and, butter]
[the, crust]
[for, the, crust]
[measured, the, flour, ,, sugar, ,, and, butter, for, the, crust]
[They, measured, the, flour, ,, sugar, ,, and, butter, for, the, crust, .]
[Her, mother]
[the, crust]
[roll, out, the, crust]
[to, roll, out, the, crust]
[how, to, roll, out, the, crust]
[showed, her, how, to, roll, out, the, crust]
[Her, mother, showed, her, how, to, roll, out, the, crust, .]
[the, apples]
[the, pie, pan]
[sliced, the, apples]
[in, the, pie, pan]
[put, them, in, the, pie, pan]
[sliced, the, apples, and, put, them, in, the, pie, pan]
[Meg, sliced, the, apples, and, put, them, in, the, pie, pan, .]
[the, pie]
[the, hot, oven]
[forty, five]
[in, the, hot, oven]
[forty, five, minutes]
[put, the, pie, in, the, hot, oven]
[waited, forty, fiv

[the, puzzles]
[with, her]
[on, the, puzzles]
[working, on, the, puzzles, with, her]
[loves, working, on, the, puzzles, with, her]
[Emma, loves, working, on, the, puzzles, with, her, .]
[was, little]
[Ms., Brown]
[small, puzzles]
[big, pieces]
[Emma, was, little]
[with, big, pieces]
[When, Emma, was, little]
[small, puzzles, with, big, pieces]
[had, small, puzzles, with, big, pieces]
[When, Emma, was, little, ,, Ms., Brown, had, small, puzzles, with, big, pieces, .]
[sixth, grade]
[large, puzzles]
[very, small]
[in, sixth, grade]
[very, small, pieces]
[is, in, sixth, grade]
[of, very, small, pieces]
[Emma, is, in, sixth, grade]
[hundreds, of, very, small, pieces]
[that, Emma, is, in, sixth, grade]
[with, hundreds, of, very, small, pieces]
[now, that, Emma, is, in, sixth, grade]
[large, puzzles, with, hundreds, of, very, small, pieces]
[on, large, puzzles, with, hundreds, of, very, small, pieces]
[work, together, on, large, puzzles, with, hundreds, of, very, small, pieces]
[But, now, th

[a, fish]
[a, nap]
[its, eyes]
[takes, a, nap]
[its, eyes, open]
[a, fish, takes, a, nap]
[keeps, its, eyes, open]
[When, a, fish, takes, a, nap]
[When, a, fish, takes, a, nap, ,, it, keeps, its, eyes, open, .,  ]
[A, fish]
[it, sleeps]
[when, it, sleeps]
[swims, when, it, sleeps]
[A, fish, swims, when, it, sleeps, !,  ]
[Some, fish]
[they, sleep]
[when, they, sleep]
[hide, when, they, sleep]
[Some, fish, hide, when, they, sleep, .]
[a, fish, naps]
[for, food]
[After, a, fish, naps]
[looks, for, food]
[After, a, fish, naps, ,, it, looks, for, food, .]
[at, trucks]
[barks, at, trucks]
[Rocky, barks, at, trucks, .,  ]
[Every, day]
[the, mail, truck]
[at, the, mail, truck]
[barks, at, the, mail, truck]
[Every, day, Rocky, barks, at, the, mail, truck, .]
[the, garbage, truck]
[Rocky, barks]
[Here, comes, the, garbage, truck]
[Here, comes, the, garbage, truck, ,, and, Rocky, barks, .]
[a, fire, truck]
[hears, a, fire, truck]
[Rocky, hears, a, fire, truck]
[When, Rocky, hears, a, fire, truck

[more, string]
[the, kite]
[very, high]
[lets, out, more, string]
[flies, very, high]
[He, lets, out, more, string]
[the, kite, flies, very, high]
[He, lets, out, more, string, and, the, kite, flies, very, high, .]
[stops, running]
[the, kite]
[the, sky]
[in, the, sky]
[dance, high, in, the, sky]
[the, kite, dance, high, in, the, sky]
[watches, the, kite, dance, high, in, the, sky]
[stops, running, and, watches, the, kite, dance, high, in, the, sky]
[Jeff, stops, running, and, watches, the, kite, dance, high, in, the, sky, .]
[Jill, and, Sam]
[the, snow]
[in, the, snow]
[stood, in, the, snow]
[Jill, and, Sam, stood, in, the, snow, .]
[the, snowflakes]
[the, snowflakes, falling]
[watched, the, snowflakes, falling]
[They, watched, the, snowflakes, falling, .,  ]
[their, mouths]
[in, their, mouths]
[caught, snowflakes, in, their, mouths]
[First, they, caught, snowflakes, in, their, mouths, .]
[the, snowflakes]
[her, sleeve]
[at, the, snowflakes]
[on, her, sleeve]
[looked, at, the, snowfla

[The, camp, leaders]
[very, helpful]
[fun, ideas]
[are, very, helpful]
[of, fun, ideas]
[lots, of, fun, ideas]
[have, lots, of, fun, ideas]
[are, very, helpful, and, have, lots, of, fun, ideas]
[The, camp, leaders, are, very, helpful, and, have, lots, of, fun, ideas, .]
[This, year]
[an, overnight, hike]
[a, lake]
[to, a, lake]
[an, overnight, hike, to, a, lake]
[for, an, overnight, hike, to, a, lake]
[went, for, an, overnight, hike, to, a, lake]
[This, year, ,, they, went, for, an, overnight, hike, to, a, lake, .,  ]
[the, way]
[two, red, foxes]
[a, field]
[On, the, way]
[in, a, field]
[playing, in, a, field]
[two, red, foxes, playing, in, a, field]
[saw, two, red, foxes, playing, in, a, field]
[On, the, way, ,, they, saw, two, red, foxes, playing, in, a, field, .]
[Everyone, froze, .]
[the, foxes]
[of, the, foxes]
[pictures, of, the, foxes]
[took, pictures, of, the, foxes]
[Brian, took, pictures, of, the, foxes, .]
[his, camera]
[with, him]
[brought, his, camera, with, him]
[he, brou

[the, garden]
[on, leaves]
[around, the, garden]
[landing, gently, on, leaves]
[flew, around, the, garden, ,, landing, gently, on, leaves]
[They, flew, around, the, garden, ,, landing, gently, on, leaves, .]
[the, plants]
[tiny, green, eggs]
[the, leaves]
[eating, the, plants]
[on, the, leaves]
[Instead, of, eating, the, plants]
[left, tiny, green, eggs, on, the, leaves]
[Instead, of, eating, the, plants, ,, they, left, tiny, green, eggs, on, the, leaves, .]
[That, way]
[the, eggs]
[the, baby, caterpillars]
[have, food]
[the, eggs, hatched]
[would, have, food]
[when, the, eggs, hatched]
[That, way, ,, when, the, eggs, hatched, ,, the, baby, caterpillars, would, have, food, .]
[A, jellyfish]
[very, strange]
[a, very, strange, creature]
[is, a, very, strange, creature]
[A, jellyfish, is, a, very, strange, creature, .]
[a, sea, animal]
[our, oceans]
[in, our, oceans]
[lives, in, our, oceans]
[that, lives, in, our, oceans]
[a, sea, animal, that, lives, in, our, oceans]
[is, a, sea, animal,

[their, old, stories]
[to, their, old, stories]
[listening, to, their, old, stories]
[in, listening, to, their, old, stories]
[very, interested, in, listening, to, their, old, stories]
[not, very, interested, in, listening, to, their, old, stories]
[Lily, 's, not, very, interested, in, listening, to, their, old, stories, .,  ]
[her, class]
[the, old, days]
[in, the, old, days]
[was, different, in, the, old, days]
[life, was, different, in, the, old, days]
[how, life, was, different, in, the, old, days]
[about, how, life, was, different, in, the, old, days]
[learning, about, how, life, was, different, in, the, old, days]
[is, learning, about, how, life, was, different, in, the, old, days]
[Now, her, class, is, learning, about, how, life, was, different, in, the, old, days, .]
[a, relative]
[one, thing]
[needed, daily]
[interview, a, relative]
[they, needed, daily]
[that, they, needed, daily]
[one, thing, that, they, needed, daily]
[find, out, one, thing, that, they, needed, daily]
[inte

[Much, like, today]
[for, drinks]
[the, drinks]
[as, sweet]
[make, them, now]
[used, for, drinks]
[we, make, them, now]
[was, used, for, drinks]
[as, we, make, them, now]
[cocoa, was, used, for, drinks]
[as, sweet, as, we, make, them, now]
[be, as, sweet, as, we, make, them, now]
[to, be, as, sweet, as, we, make, them, now]
[prepared, to, be, as, sweet, as, we, make, them, now]
[were, n't, prepared, to, be, as, sweet, as, we, make, them, now]
[the, drinks, were, n't, prepared, to, be, as, sweet, as, we, make, them, now]
[Much, like, today, ,, cocoa, was, used, for, drinks, ,, but, the, drinks, were, n't, prepared, to, be, as, sweet, as, we, make, them, now, .]
[Ancient, cocoa, products]
[chili, or, vanilla]
[with, chili, or, vanilla]
[sometimes, seasoned, with, chili, or, vanilla]
[bitter, ,, sometimes, seasoned, with, chili, or, vanilla]
[were, bitter, ,, sometimes, seasoned, with, chili, or, vanilla]
[Ancient, cocoa, products, were, bitter, ,, sometimes, seasoned, with, chili, or, va

[her, Spanish, language, and, cooking, skills]
[limited, screen, time]
[improving, her, Spanish, language, and, cooking, skills]
[on, limited, screen, time]
[to, improving, her, Spanish, language, and, cooking, skills]
[get, by, on, limited, screen, time]
[addition, to, improving, her, Spanish, language, and, cooking, skills]
[to, get, by, on, limited, screen, time]
[In, addition, to, improving, her, Spanish, language, and, cooking, skills]
[learning, to, get, by, on, limited, screen, time]
[is, learning, to, get, by, on, limited, screen, time]
[In, addition, to, improving, her, Spanish, language, and, cooking, skills, ,, she, is, learning, to, get, by, on, limited, screen, time, .]
[A, ship]
[the, sea]
[across, the, sea]
[sailing, across, the, sea]
[is, sailing, across, the, sea]
[A, ship, is, sailing, across, the, sea, .]
[The, water]
[get, rough]
[the, ship]
[to, sway]
[to, get, rough]
[begins, to, sway]
[starts, to, get, rough]
[the, ship, begins, to, sway]
[The, water, starts, to,

In [ ]:
0,1,0,0,0,0,0